<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Segment mode and comparison with basic usage

This notebook compares the basic usage (Option 1) with the segment mode (Option 2).
It shows how to use segment mode and returns information about the number of segments for a given memory in MegaBytes.

Note: The segment mode can be combined with a Region Of Interest (ROI). Only a standard example without ROI is provided in the following.

#### Imports and external functions

In [ ]:
from pathlib import Path
from pprint import pprint
import numpy as np

In [ ]:
from snippets.utils import *

# Pandora2D execution options with state machine

Without segment mode.

#### Imports of pandora2d

In [ ]:
# Load pandora2d imports
import pandora2d
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import create_datasets_from_inputs

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

Provide output directory to write results

In [ ]:
output_dir = Path.cwd() / "output"
# If necessary, create output dir
output_dir.mkdir(exist_ok=True,parents=True)

Convert input data to dataset

In [ ]:
input_config = {
    "left": {"img": img_left_path, "nodata": np.nan},
    "right": {"img": img_right_path, "nodata": np.nan},
    "col_disparity": {"init": 0, "range": 2},
    "row_disparity": {"init": 0, "range": 2},
}

In [ ]:
img_left, img_right = create_datasets_from_inputs(input_config=input_config)

`create_datasets_from_inputs` returns a namedTuple so we could have used:

```python
image_datasets = create_datasets_from_inputs(input_config=input_config)
```

and called:

 `image_datasets.left` or `image_datasets.right` instead of `img_left` and `img_right`.

In [ ]:
plot_two_images(img_left["im"].data,
                img_right["im"].data,
                "Left image",
                "Right image",
                output_dir)

# Option 1 : trigger all the steps of the machine at ones

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": "data/left.tif",
            "nodata": "NaN",
        },
        "right": {
            "img": "data/right.tif",
        },
        "col_disparity": {"init": 0, "range": 2},
        "row_disparity": {"init": 0, "range": 2},
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc_python",
            "subpix": 4,
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -5
        },
        "refinement" : {
            "refinement_method": "dichotomy",
            "iterations": 2,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/basic_usage_for_comparison"
    },
}

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_conf(user_cfg, pandora2d_machine)

In [ ]:
pipeline_cfg = checked_cfg['pipeline']
pprint(pipeline_cfg)

#### Trigger all the steps of the machine at ones

In [ ]:
dataset_basic, _ = pandora2d.run_pandora2d(
    pandora2d_machine,
    checked_cfg
    )

Visualize output disparity map

In [ ]:
plot_two_images(dataset_basic["row_map"].data,
                dataset_basic["col_map"].data,
                "Row disparity map",
                "Columns disparity map",
                output_dir,
                cmap=pandora_cmap())

Visualize correlation score 

In [ ]:
plot_image(dataset_basic["correlation_score"].data, "Correlation score", output_dir, cmap=pandora_cmap())

# Option 2 : Use segment mode for limited memory usage

The implementation of Pandora2D allows to use a segment mode.

This mode requires to provide the available memory and Pandora2D computes automatically the size of a segment.

It provides a list of segments (by their ROI) and computes segment per segment, with a free of the memory at each step.

#### Instantiate the machine

In [ ]:
del(pandora2d_machine)

pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": "data/left.tif",
            "nodata": "NaN",
        },
        "right": {
            "img": "data/right.tif",
        },
        "col_disparity": {"init": 0, "range": 2},
        "row_disparity": {"init": 0, "range": 2},
    },
    "segment_mode": {
        "enable": True,
        "memory_per_work": 300
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc_python",
            "subpix": 4,
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -5
        },
        "refinement":{
            "refinement_method": "dichotomy",
            "iterations": 2,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/introduction_and_basic_usage_option_2"
    },
}

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_conf(user_cfg, pandora2d_machine)

In [ ]:
pipeline_cfg = checked_cfg['pipeline']
pprint(pipeline_cfg)

Get list of ROIs for prints only

In [ ]:
from pandora2d.memory_estimation import segment_image_by_rows

list_of_roi = segment_image_by_rows(
        checked_cfg, pandora2d_machine.margins_disp.global_margins, pandora2d_machine.margins_img.global_margins
    )

In [ ]:
print("Number of ROIs:", len((list_of_roi)))

#### Run pandora2d with segment mode

In [ ]:
dataset_segment, _ = pandora2d.run_pandora2d_segment_mode(
    pandora2d_machine,
    checked_cfg
    )

In [ ]:
plot_two_images(dataset_segment["row_map"].data,
                dataset_segment["col_map"].data,
                "Row disparity map",
                "Columns disparity map",
                output_dir,
                cmap=pandora_cmap())

#### Differences with/without segment mode

Must return "0", as there are no differences between outputs.

In [ ]:
diff_row = abs(dataset_basic["row_map"].data - dataset_segment["row_map"].data)
diff_col = abs(dataset_basic["col_map"].data - dataset_segment["col_map"].data)
diff_corr = abs(dataset_basic["correlation_score"].data \
                - dataset_segment["correlation_score"].data)

In [ ]:
print("Sum of differences for rows:", diff_row.sum())
print("Sum of differences for cols:", diff_col.sum())
print("Sum of differences for scores:", np.nansum(diff_corr))